# Importing Dependencies

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Loading Data

In [2]:
with open('/kaggle/input/text-data/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Tokenize and Sequencing

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
num_words = len(tokenizer.word_index) + 1

In [4]:
num_words

8200

# Sequencing , forming n-grams

In [5]:
input_sequences = []
for line in text.split('\n'):
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokens)):
        ngram_sequence = tokens[:i+1]
        input_sequences.append(ngram_sequence)

# Padding

In [6]:
max_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_len, padding='pre'))

In [7]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=num_words))

# Model Building

In [9]:
model = Sequential()
model.add(Embedding(num_words, 100, input_length=max_len-1))
model.add(LSTM(150))
model.add(Dense(num_words, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Model Training

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 40s 11ms/step - loss: 6.2214 - accuracy: 0.0776
Epoch 2/100
3010/3010 [==============================] - 20s 7ms/step - loss: 5.4960 - accuracy: 0.1257
Epoch 3/100
3010/3010 [==============================] - 19s 6ms/step - loss: 5.1162 - accuracy: 0.1476
Epoch 4/100
3010/3010 [==============================] - 19s 6ms/step - loss: 4.7858 - accuracy: 0.1662
Epoch 5/100
3010/3010 [==============================] - 19s 6ms/step - loss: 4.4818 - accuracy: 0.1841
Epoch 6/100
3010/3010 [==============================] - 18s 6ms/step - loss: 4.1980 - accuracy: 0.2041
Epoch 7/100
3010/3010 [==============================] - 19s 6ms/step - loss: 3.9284 - accuracy: 0.2286
Epoch 8/100
3010/3010 [==============================] - 18s 6ms/step - loss: 3.6681 - accuracy: 0.2580
Epoch 9/100
3010/3010 [==============================] - 19s 6ms/step - loss: 3.4278 - accuracy: 0.2909
Epoch 10/100
3010/3010 [==============================] - 19s 6

# Prediction

In [12]:
seed_text = "I will leave if they"
next_words = 10

for _ in range(next_words):
    tokens = tokenizer.texts_to_sequences([seed_text])[0]
    tokens = pad_sequences([tokens], maxlen=max_len-1, padding='pre')
    predicted = np.argmax(model.predict(tokens), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


1/1 [==============================] - 0s 18ms/step


In [13]:
seed_text

'I will leave if they are very incomplete how you did it seems to me'